In [1]:
# Dependencies
import pandas as pd
import os

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine,inspect, MetaData
from sqlalchemy.orm import Session

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb

<function pymysql.install_as_MySQLdb>

In [2]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

# Create Classes
class Measurements(Base):
    __tablename__="Measurements"
    Index = Column(Integer, primary_key=True)
    Station = Column(String(50))
    Date = Column(String(15))
    Prcp = Column(Float)
    Tobs= Column(Integer)
    
class Stations(Base):
    __tablename__="Stations"
    Index = Column(Integer, primary_key=True)
    Station = Column(String(50))
    Name = Column(String(255))
    Latitude = Column(Float)
    Longitude = Column(Float)
    Elevation = Column(Float)

In [3]:
# Create Database Connection
# ----------------------------------
# Creates a connection to DB using the MySQL Connect Engine
engine=create_engine("sqlite:///..\Resources\hawaii.sqlite")
conn=engine.connect()

# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

# Create a Session Object to Connect to DB
# ----------------------------------
# Session is a temporary binding to our DB
session = Session(bind=engine)

In [4]:
# Creating the filepaths
filepath1=os.path.join("..","Resources","clean_hawaii_measurements.csv")
filepath2=os.path.join("..","Resources","clean_hawaii_stations.csv")
# Reading the csv files and converting to dataframe
Hawaii_Measurement=pd.read_csv(filepath1)
print("Completed Reading File:%s"%filepath1)
Hawaii_Station=pd.read_csv(filepath2)
print("Completed Reading File:%s"%filepath2)

Completed Reading File:..\Resources\clean_hawaii_measurements.csv
Completed Reading File:..\Resources\clean_hawaii_stations.csv


In [17]:
# Dropping the table if it had been created before.
# This step is necessary only when running multiple times and is not required in the production environment
try:
    conn.execute("DROP TABLE Measurements")
    print("Dropping Table: Measurements as it already exists")
    conn.execute("DROP TABLE Stations")
    print("Dropping Table: Stations as it already exists")
except:
    pass

Dropping Table: Measurements as it already exists
Dropping Table: Stations as it already exists


In [5]:
# Create the tables in sqlite Hawaii db
Hawaii_Station.to_sql("Stations",con=conn,if_exists = "append")
Hawaii_Measurement.to_sql("Measurements",con=conn,if_exists = "append")
session.commit()


In [6]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)
# Check if the table has been created within the database
inspector.get_table_names()

['Measurements', 'Stations']

In [7]:
# Printing the first 10 rows of Measurement table
conn.execute("SELECT * FROM Measurements LIMIT(10)").fetchall()

[(0, 'USC00519397', '2010-01-01', 0.08, 65),
 (1, 'USC00519397', '2010-01-02', 0.0, 63),
 (2, 'USC00519397', '2010-01-03', 0.0, 74),
 (3, 'USC00519397', '2010-01-04', 0.0, 76),
 (4, 'USC00519397', '2010-01-06', 0.16064353974479206, 73),
 (5, 'USC00519397', '2010-01-07', 0.06, 70),
 (6, 'USC00519397', '2010-01-08', 0.0, 64),
 (7, 'USC00519397', '2010-01-09', 0.0, 68),
 (8, 'USC00519397', '2010-01-10', 0.0, 73),
 (9, 'USC00519397', '2010-01-11', 0.01, 64)]

In [8]:
# Printing the first 10 rows of Station table
conn.execute("SELECT * FROM Stations LIMIT(10)").fetchall()

[(0, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (1, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (2, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (3, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (4, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (5, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (6, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (7, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (8, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [9]:
conn.close()